[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gcmatos/python-para-geociencias/blob/master/notebooks/3.2%20EDA%20Erupções%20Vulcânicas%20do%20Holoceno.ipynb)

Ctrl/Cmd + click para abrir em uma nova aba do navegador web e utilizar o Google Colab para rodar o tutorial.

# Análise Exploratória de Dados
__Vulcanismo no Holoceno do "The Smithsonian Institution"__

Fonte: [https://www.kaggle.com/smithsonian/volcanic-eruptions](https://www.kaggle.com/smithsonian/volcanic-eruptions)



## Configuração de ambiente


### Instalação de dependências

In [ ]:
# Instalar GeoPandas e dependências
!pip install shapely fiona six geopandas

### Importar bibliotecas

In [ ]:
# Processamento
import numpy as np
import pandas as pd
import geopandas as gpd
import cartopy
import cartopy.crs as ccrs
from shapely.geometry import Point

# Visualizacao
import matplotlib.pyplot as plt
import seaborn as sns

# Filtrar alertas
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline

In [ ]:
# Matplotlib style
plt.style.use('default')

### Subir arquivos
Subir o arquivo **`volcanic-eruptions-database.csv`**


In [ ]:
# Subir arquivo

In [ ]:
# Listar arquivos no diretório de trabalho
%ls

### Importação dos dados

In [ ]:
df = pd.read_csv("../dados/volcanic-eruptions-database.csv")

## Auditando dados
> [Descrição das variáveis no web site do Smithsonian Institution](https://volcano.si.edu/)

Trata-se de uma série ou uma tabela de atributos?

In [ ]:
df.head()
# df.tail()

Quais são os nomes (*labels*) das variáveis e dos índices?

In [ ]:
df.columns

> Que tipos de objetos o pandas reconheceu durante a importação dos dados?

> Há variáveis numéricas e categóricas?

> Colunas com datas foram reconhecidas como `pandas.to_datetime` durante a importação?

In [ ]:
df.info()

Resumo estatístico das variáveis numéricas:

In [ ]:
df.describe()

### Valores nulos
Como é a distribuição de valores nulos no dataset?

In [ ]:
df.isnull().sum().sort_values()

### Variáveis categóricas

Como estão distribuídas as categorias das variáveis categóricas?

In [ ]:
df['Type'].value_counts()

### Série de tempo
Conferindo os valores de tempo informados na coluna 'Last Known Eruption'.

In [ ]:
df['Last Known Eruption'].dtype
# df['Last Known Eruption'].sample(5)
# df['Last Known Eruption'].replace({'Unknown': np.nan}, inplace=True)

In [ ]:
# Separar os valores de tempo do período (BCE, CE)
df[['Year', 'Period']] = df['Last Known Eruption'].str.split(' ', expand=True)

In [ ]:
df['Year'].sample(5)

In [ ]:
df['Period'].unique()

In [ ]:
# Converter valores em números
df['Year'] = pd.to_numeric(df['Year'])
df['Year'].sample(5)

In [ ]:
# Converter valores de 'Year' para CE > 0, BCE < 0
mask = df['Period'] == 'BCE'
df['Year'][mask] = -df['Year']

### Converter DataFrame em GeoDataFrame

Converter as colunas Latitude e Longitude em geometria `shapely` para converter o dataset em `GeoDataFrame`.

In [ ]:
# Criar coluna 'geometry' com lista concatenada de Lon e Lat
df['geometry'] = list(zip(df.Longitude, df.Latitude))
# Converter lista em objeto shapely.geometry.Point
df['geometry'] = df['geometry'].apply(Point)
# Converter DataFrame em GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
# Inserir código para sistema de coordenadas WGS84
gdf.crs = {'init' :'epsg:4326'}

## Visualização

### Pré-visualização

In [ ]:
# df.plot();
gdf.plot();

In [ ]:
sns.pairplot(gdf, diag_kind='kde', plot_kws={'alpha': 0.2});

### Valores nulos

In [ ]:
# Plotar a soma ordenada dos valores nulos por variável
gdf.isnull().sum().sort_values().plot.barh();

### Variáveis contínuas

In [ ]:
def line_plot(data):
    '''Plotar série de valores ordenados em gráfico de linha'''
    series = pd.Series(data).value_counts().sort_index()
    return series.plot()


def bar_plot(data):
    '''Plotar série de valores ordenados em gráfico de barras'''
    series = pd.Series(data).value_counts().sort_values()
    return series.plot(kind='bar')


def barh_plot(data):
    '''Plotar série de valores ordenados em gráfico de barras'''
    series = pd.Series(data).value_counts().sort_values()
    return series.plot(kind='barh')

In [ ]:
gdf['Elevation (Meters)'].plot();
# line_plot(gdf['Elevation (Meters)']);

In [ ]:
gdf['Elevation (Meters)'].plot.hist();

### Variáveis categóricas

Qual é a distribuição anual de atividades vulcânicas neste data set? 

A variação observada é resultado da variação de atividade vulcânica no planeta ou é um efeito da amostragem?

In [ ]:
plt.figure(figsize=(12, 5))

line_plot(gdf["Year"])
# bar_plot(gdf["Year"])

plt.ylabel("Frequência")
plt.title("Atividade vulcânica");

In [ ]:
plt.figure(figsize=(5, 10))
barh_plot(gdf["Region"])
# gdf["Region"].value_counts().sort_values().plot.barh()
plt.ylabel("Região")
plt.title("Regiões com mais vulcanismo durante o Holoceno")

Qual é a distribuição de atividades vulcânicas por país? 

A diferença encontrada é um fator geológico ou de desenvolvimento científico?

In [ ]:
plt.figure(figsize=(5, 25))
barh_plot(gdf["Country"])
# gdf["Country"].value_counts().sort_values().plot.barh()
plt.ylabel("País")
plt.title("Atividade vulcânica por país")
plt.show()

Quais são os tipos de evidências de vulcanismo neste dataset?

In [ ]:
# gdf["Activity Evidence"].value_counts().sort_values().plot.barh()
barh_plot(gdf["Activity Evidence"])
plt.ylabel("Evidências")
plt.title("Evidência de atividade vulcânica")
plt.show()

Quais são os tipos de rochas dominantes no dataset?

In [ ]:
# gdf["Dominant Rock Type"].value_counts().sort_values().plot.barh()
barh_plot(gdf["Dominant Rock Type"])
plt.ylabel("Tipos de rocha")
plt.title("Tipo de rocha dominante")
plt.show()

Quais são os regimes tectônicos?

In [ ]:
# gdf["Tectonic Setting"].value_counts().sort_values().plot.barh()
barh_plot(gdf["Tectonic Setting"])
plt.ylabel("Regime tectônico")
plt.title("Regime tectônico")
plt.show()

### Correlações entre variáveis

In [ ]:
# Calcular a matrix de correlação
corr = gdf.corr()
# Plotar a matriz de correlação
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

### Distribuição espacial

In [ ]:
# Carregar o shape com os contornos dos países do GeoPandas.dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

Como estão distribuídos as erupções vulcânicas em termos de coordenadas geográficas?

In [ ]:
fig = plt.figure(figsize=(15, 6))
# Criar instancia Cartopy
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
# Adicionar raster de terreno
ax.stock_img()
# Plotar os contornos dos países com GeoPandas
world.plot(ax=ax, facecolor='none', linewidth=0.5, edgecolor='black')
# Plotar os registros dos terremotos
gdf.plot(ax=ax, color='red', markersize=10, alpha=0.4)
plt.show()

Qual é a frequencia de erupções vulcânicas por localidade?

In [ ]:
fig = plt.figure(figsize=(15, 6))
# Criar instancia Cartopy
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
# Adicionar raster de terreno
ax.stock_img()
# Plotar os contornos dos países com GeoPandas
world.plot(ax=ax, facecolor='none', linewidth=0.5, edgecolor='black')
# Plotar os registros dos terremotos
gdf.plot(
    ax=ax, column='Number', markersize=10, alpha=0.4, cmap='hot', legend=True)
plt.show()